# Testing irrep construction

Given state $\ket{v}$, 1D irrep $T$ of order $N$ (i.e. such that 
$$
\{\ket{v}, T\ket{v}, T^2\ket{v}, ... , T^{N-1}\ket{v}\}
$$
is a subspace of dimension $N$), straightforward to construct Bloch waves

$$
\ket{k} := \sum_{n=0}^{N-1} e^{i k n} T^{n}\ket{v}
$$

where $k = 2\pi m/N$ for some integer $m$.



In [202]:
import numpy as np
import bit_tools
import bisect
from sympy.combinatorics import PermutationGroup, Permutation
from sortedcontainers import SortedDict


In [164]:

def find_word(G, perm):
    """Finds the word representation of perm in terms of G.generators"""
    from collections import deque
    
    # Identity permutation
    identity = Permutation(list(range(len(perm.array_form))))
    
    # BFS queue: stores (current permutation, word leading to it)
    queue = deque([(identity, [])])
    visited = {identity: []}
    
    generators = G.generators + [g**-1 for g in G.generators]  # Include inverses
    gen_indices = [i+1 for i in range(len(G.generators))] + [-i-1 for i in range(len(G.generators))]

    while queue:
        current_perm, word = queue.popleft()

        # If we found the target permutation, return the word
        if current_perm == perm:
            return word
        
        for gen, index in zip(generators, gen_indices):
            new_perm = current_perm * gen
            
            if new_perm not in visited:
                visited[new_perm] = word + [index]  # Extend the word
                queue.append((new_perm, word + [index]))
    
    return None  # Should never happen in a well-defined group

In [146]:
def multirange(*args):
    return itertools.product(*[range(n) for n in args])

def group_power(generators, N):
    h = generators[0]**0
    for g, n in zip(generators, N):
        h *= g**n
    return h
    

In [194]:

def partition_basis_Abelian(blist, group:PermutationGroup, verify_basis=True):
    """
    @param blist -> a list of basis vectors, represented in Sz basis as bitstrings
    @param group -> the symmetry group
    @param verify_basis -> enables checks that the basis respects the symmetries

    @return a SortedDict, keys are basis states, values are of form (seed_state_idx, (n1,n2,n3))
        to indicate that the idx'th
    """
    orbit_index = SortedDict()
    
    covered_elems = [] # the basis elements already in an orbit
    assert group.is_abelian

    gen_orders = [g.order() for g in group.generators]

    seed_state_list = []

    num_orbits = 0
    idx = 0
    while idx < len(blist):
        seed_state = blist[idx]
        

        J = bisect.bisect_left(covered_elems, seed_state)
        if J < len(covered_elems) and covered_elems[J] == seed_state:
            # we have already hit this
            idx+= 1
            continue

        seed_state_list.append(seed_state)

        # curr_orbit = []
        
        for N in multirange(*gen_orders):
            g = group_power(group.generators, N)
            state = bit_tools.bitperm(g.array_form, seed_state)
            idx_to_insert = bisect.bisect_left(covered_elems, state)

            if verify_basis:
                J = bisect.bisect_left(blist, state)
                if blist[J] != state:
                    raise ValueError("Basis is not closed under the symmetry operation")
            
            if idx_to_insert >= len(covered_elems) or covered_elems[idx_to_insert] != state:
                covered_elems.insert(idx_to_insert, state)
                orbit_index[state] = (num_orbits, N)
                # curr_orbit.append((N, state))
                # sector_dims[N] += 1
            
        

        # orbits.append(curr_orbit)
        num_orbits += 1
        idx+= 1

    return orbit_index, seed_state_list

In [216]:
Lx = 4
Ly = 5


def make_basis(N):
    basis = []
    for J in range(1<<N):
        if J.bit_count() == N//2:
            basis.append(J)
    basis.sort()
    return basis

%time basis = make_basis(Lx*Ly)

def make_transl_ops(Lx, Ly):
    Tx = np.zeros(Lx*Ly)
    Ty = np.zeros(Lx*Ly)
    
    for ix in range(Lx):
        for iy in range(Ly):
            Tx[Ly*ix+iy] = Ly*((ix+1)%Lx)+iy
            Ty[Ly*ix+iy] = Ly*ix + ((iy+1)%Ly)
    return Tx, Ty

%time Tx, Ty = make_transl_ops(Lx, Ly)

G = PermutationGroup(Permutation(Tx), Permutation(Ty))



CPU times: user 63.9 ms, sys: 9.15 ms, total: 73.1 ms
Wall time: 99.6 ms
CPU times: user 18 μs, sys: 7 μs, total: 25 μs
Wall time: 25.7 μs


In [212]:
%%time
basis_partition, seeds = partition_basis_Abelian(basis, G)

CPU times: user 5.63 s, sys: 25.6 ms, total: 5.65 s
Wall time: 5.65 s


In [200]:
# check that this works as expected
for state in basis_partition:
    J, N = basis_partition[state]
    s2 = bit_tools.bitperm(group_power(G.generators, N).array_form, seeds[J])
    assert state == s2, f"{state:09b} {s2:09b} seed {seeds[J]:09b} ^ {N}"

[np.int64(3)]